<a href="https://colab.research.google.com/github/Dynamicduck/Diffusion-Reaction-Animations/blob/master/Diffusion_Reaction_Animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gray Scott Reaction Diffusion Animation Generator
This notebook can be used to generate two-component diffusion-reaction animations.

Run each code cell in order. Then you can play with the settings and run the final cell to generate different animations.

## Creating functions

There is currently a bug relating to matplotlib which will sometimes return an "AttributeError" when this cell is run. The code will work nonetheless. https://github.com/ipython/ipython/issues/10873

In [98]:
#@title
 
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from IPython.display import HTML
 
animation.rcParams['animation.embed_limit'] = 2**128
 
 
def initialise(x, y, xb, yb, Ub, Vb, Un, Vn):   #takes initial conditions and creates the initial populations in a simulation region    
    U = np.ones((x, y)) #creates simulation region filled with U
    V = np.zeros((x, y))
    
    xb = int(xb/2) #creates a central box of V
    yb = int(yb/2)
    xlow = int(x/2) - xb
    xhigh = int(x/2) + xb
    ylow = int(y/2) - yb
    yhigh = int(y/2) + yb
    U[xlow:xhigh, ylow:yhigh] = Ub
    V[xlow:xhigh, ylow:yhigh] = Vb
    
    U += Un * np.random.random((x, y)) #adds noise
    V += Vn * np.random.random((x, y))
    
    return U, V
 
 
def F(U, V, f): #functions for Gray-Scott model
    return -U*V**2 + f*(1 - U)
 
def G(U, V, f, k):
    return U*V**2 - (f + k)*V
 
 
def laplacian(a): #finds laplacians of the simulation region (and uses/enforces periodic boundary conditions)
    l = -4*a
    l += np.roll(a, (0,-1), (0,1))
    l += np.roll(a, (0,+1), (0,1))
    l += np.roll(a, (-1,0), (0,1))
    l += np.roll(a, (+1,0), (0,1))
    return l
 
 
def runge(U, V, dt, Du, Dv, f, k): #Runge-Kutta 4 method for evolving the simulation
    k1 = k2 = k3 = k4 = j1 = j2 = j3 = j4 = np.zeros(U.shape)
    
    k1 = dt*(F(U, V, f) + Du * laplacian(U))
    j1 = dt*(G(U, V, f, k) + Dv * laplacian(V))
    
    k2 = dt*(F(U+k1, V+j1, f) + Du * laplacian(U+k1))
    j2 = dt*(G(U+k1, V+j1, f, k) + Dv * laplacian(V+j1))
    
    k3 = dt*(F(U+k2, V+j2, f) + Du * laplacian(U+k2))
    j3 = dt*(G(U+k2, V+j2, f, k) + Dv * laplacian(V+j2))
    
    k4 = dt*(F(U+k3, V+j3, f) + Du * laplacian(U+k3))
    j4 = dt*(G(U+k3, V+j3, f, k) + Dv * laplacian(V+j3))
    
    U = U + k1/6 + k2/3 + k3/3 + k4/6
    V = V + j1/6 + j2/3 + j3/3 + j4/6
    
    return U, V
 
 
def run(U, V, dt, t, Du, Dv, f, k, fps, frames, Vb): #finally runs simulation from initial conditions, using population variables passed and outputs a gif
    plt.close("all")
    fig = plt.figure()
    plt.axis("off")
    
    tt = int(t/dt)
    ims = []
    dtpf = int(tt/frames)
    
    bar = widgets.FloatProgress(
      value=0,
      min=0,
      max=1,
      description='Processing:',
      bar_style='info',
      orientation='horizontal'
      )
    display(bar)
 
    for i in range(0, tt):
        U, V = runge(U, V, dt, Du, Dv, f, k)
        
        if i % int(dtpf) == 0:
            im = plt.imshow(V, vmin=0, vmax=Vb)
            ims.append([im])
        
        if i % 100 == 0:
            bar.value = i/tt
    
    bar.value = 1
    print("Creating animation...")
    
    ani = animation.ArtistAnimation(fig, ims, interval=1000/fps, blit=True)
    return ani

## Simulation Settings

In [99]:
#@title
 
y = widgets.BoundedIntText(
    value=150,
    min=0,
    max=1000,
    step=1,
    description='Simulation x:'
)
display(y)
 
x = widgets.BoundedIntText(
    value=150,
    min=0,
    max=1000,
    step=1,
    description='Simulation y:'
)
display(x)
 
dt = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    description='Time step:'
)
display(dt)
 
t = widgets.BoundedIntText(
    value=9000,
    min=0,
    max=100000,
    step=1,
    description='Time steps:'
)
display(t)
 
Un = widgets.FloatSlider(
    value=0,
    min=0,
    max=0.5,
    step=0.01,
    description='U (Prey) noise:'
)
display(Un)
 
Vn = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=0.5,
    step=0.01,
    description='V (Predator) noise:'
)
display(Vn)

BoundedIntText(value=150, description='Simulation x:', max=1000)

BoundedIntText(value=150, description='Simulation y:', max=1000)

IntSlider(value=1, description='Time step:', max=10, min=1)

BoundedIntText(value=9000, description='Time steps:', max=100000)

FloatSlider(value=0.0, description='U (Prey) noise:', max=0.5, step=0.01)

FloatSlider(value=0.1, description='V (Predator) noise:', max=0.5, step=0.01)

## Population Injection Settings

In [100]:
#@title
 
yb = widgets.BoundedIntText(
    value=10,
    min=0,
    max=1000,
    step=1,
    description='Injection x:'
)
display(yb)
 
xb = widgets.BoundedIntText(
    value=10,
    min=0,
    max=1000,
    step=1,
    description='Injection y:'
)
display(xb)
 
Ub = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.05,
    description='U (Prey) in injection:'
)
display(Ub)
 
Vb = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=1,
    step=0.05,
    description='V (Predators) in injection:'
)
display(Vb)

BoundedIntText(value=10, description='Injection x:', max=1000)

BoundedIntText(value=10, description='Injection y:', max=1000)

FloatSlider(value=0.5, description='U (Prey) in injection:', max=1.0, step=0.05)

FloatSlider(value=0.5, description='V (Predators) in injection:', max=1.0, step=0.05)

## Gray-Scott Variables

Some interesting Gray-Scott variable combos to try:

*   List item
*   List item


In [101]:
#@title
 
Du = widgets.FloatSlider(
    value=0.13,
    min=0,
    max=0.25,
    step=0.01,
    description='U (Prey) difussion:'
)
display(Du)
 
Dv = widgets.FloatSlider(
    value=0.06,
    min=0,
    max=0.25,
    step=0.01,
    description='V (Predator) Diffusion:'
)
display(Dv)
 
f = widgets.FloatSlider(
    value=6,
    min=0,
    max=10,
    step=0.1,
    description='Feed rate E-2:'
)
display(f)
 
k = widgets.FloatSlider(
    value=6.2,
    min=0,
    max=10,
    step=0.1,
    description='Kill rate E-2:'
)
display(k)

FloatSlider(value=0.13, description='U (Prey) difussion:', max=0.25, step=0.01)

FloatSlider(value=0.06, description='V (Predator) Diffusion:', max=0.25, step=0.01)

FloatSlider(value=6.0, description='Feed rate E-2:', max=10.0)

FloatSlider(value=6.2, description='Kill rate E-2:', max=10.0)

## Animation Settings

In [102]:
#@title
 
fps = widgets.IntSlider(
    value=15,
    min=1,
    max=60,
    description='Fps:'
)
display(fps)
 
frames = widgets.BoundedIntText(
    value=150,
    min=0,
    max=100000,
    step=1,
    description='Frames:'
)
display(frames)

IntSlider(value=15, description='Fps:', max=60, min=1)

BoundedIntText(value=150, description='Frames:', max=100000)

## Run Simulation and Create Animation
Edit the settings above and then run the cell below. Running the cells above will reset the respective settings to their defaults.

There is currently a bug relating to matplotlib which will sometimes return an "AttributeError" when this cell is run. The code will work nonetheless. https://github.com/ipython/ipython/issues/10873


In [103]:
#@title
 
dtpf = (t.value/dt.value)/frames.value
print("Time steps per frame: %.3g" % dtpf)
 
U, V = initialise(x.value, y.value, xb.value, yb.value, Ub.value, Vb.value, Un.value, Vn.value)
ani = run(U, V, dt.value, t.value, Du.value, Dv.value, f.value*10**-2, k.value*10**-2, fps.value, frames.value, Vb.value)
print("Displaying animation...")
HTML(ani.to_html5_video())

Time steps per frame: 60


<IPython.core.display.Javascript object>

FloatProgress(value=0.0, bar_style='info', description='Processing:', max=1.0)

Creating animation...
Displaying animation...
